In [1]:
!export LC_ALL=C.UTF-8
!export LANG=C.UTF-8
!export LANGUAGE=C.UTF-8

In [ ]:
!pip install -U torch torchvision torchaudio
!pip install -U ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 766.7/766.7 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 104.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 76.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.1/150.1 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 90.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#!rsync -ah --progress /content/drive/MyDrive/dataset/ /content/dataset/
#超重要！！！！！
#ローカルPCでdatasetをdataset.zipに圧縮してMyDriveへコピーしてください。

# ZIPファイルをGoogle DriveからColabへコピー
!rsync -ah --progress /content/drive/MyDrive/dataset.zip /content/

# ZIPファイルを解凍
!unzip -o /content/dataset.zip -d /content/

# (任意) 解凍後のZIPファイルを削除してディスクを節約
!rm /content/dataset.zip

sending incremental file list
data.yaml
          1.43K 100%    0.00kB/s    0:00:00 (xfr#1, ir-chk=2389/2391)
labels/
labels/train/8496046d-Ch120250215003357.txt
          2.46K 100%    0.00kB/s    0:00:00 (xfr#2, to-chk=1589/5968)
labels/train/84ab0bad-Ch120241207080423.txt
          1.64K 100%    5.07kB/s    0:00:00 (xfr#3, to-chk=1588/5968)
labels/train/84bf2a37-Ch120241210092726.txt
            818 100%    1.16kB/s    0:00:00 (xfr#4, to-chk=1587/5968)
labels/train/85dfc07e-Ch120250126114322.txt
            484 100%    0.49kB/s    0:00:00 (xfr#5, to-chk=1585/5968)
labels/train/879b5517-Ch120241224125351.txt
          1.14K 100%    0.90kB/s    0:00:01 (xfr#6, to-chk=1574/5968)
labels/train/87d4ec3c-Ch120241207120901.txt
          1.79K 100%    0.00kB/s    0:00:00 (xfr#7, to-chk=1572/5968)
labels/train/87dcfbf9-Ch120250130110221.txt
          2.86K 100%   11.08kB/s    0:00:00 (xfr#8, to-chk=1571/5968)
labels/train/87e74ab1-Ch120241210093637.txt
            654 100%    1.36kB/s    0:00

In [ ]:
model_path = "/content/drive/MyDrive/yolov8s.pt"
# YAMLファイルのパス
data_yaml = "/content/dataset/data.yaml"

In [ ]:
import yaml


# YAMLファイルを読み込む
with open(data_yaml, "r") as file:
    data = yaml.safe_load(file)

# 変更したいパス
data["test"] = "./images/test"
data["train"] = "./images/train"
data["val"] = "./images/val"

# YAMLファイルを書き戻す
with open(data_yaml, "w") as file:
    yaml.safe_dump(data, file, default_flow_style=False)

print("data.yaml のパスを更新しました。")

data.yaml のパスを更新しました。


In [ ]:
import os
from ultralytics import YOLO
import numpy as np


def train_yolov8(model_path, data_yaml, epochs, batch_size, img_size, output_dir, iou_threshold, mosaic, mixup):
    """
    Train YOLOv8 model with enhanced settings for small overlapping object detection.
    """
    print(f"Loading model from {model_path}")
    model = YOLO(model_path)

    # Train the model with optimized settings
    print(f"Training YOLOv8 with {epochs} epochs...")
    model.train(
        data=data_yaml,
        epochs=epochs,
        batch=batch_size,
        imgsz=img_size,
        project=output_dir,
        name="yolov8_training",
        iou=iou_threshold,  # Adjust IoU threshold for better object separation
        mosaic=mosaic,  # Enhance data augmentation with mosaic images
        mixup=mixup,  # Mixup augmentation for diverse training
        degrees=180,  # Rotation augmentation
    )

    print(f"Training complete. Results saved to {output_dir}")

# Colab 用にパラメータを手動で指定
model_path = "/content/drive/MyDrive/yolov8s.pt"  # モデルのパス (Google Drive に保存している場合は適宜変更)
#data_yaml = "/content/drive/MyDrive/MaiNDLab/dataset/data.yaml"  # データセットの YAML ファイル
epochs = 200  # エポック数
batch_size = 64  # バッチサイズ
img_size = 960  # 画像サイズ
iou_threshold = 0.3  # IoU 閾値
mosaic = 1.0  # モザイクの確率
mixup = 0.2  # Mixup の確率
output_dir = "runs_experiment"  # 出力ディレクトリ

# 出力ディレクトリを作成
os.makedirs(output_dir, exist_ok=True)

# YOLOv8 の学習実行
train_yolov8(
    model_path=model_path,
    data_yaml=data_yaml,
    epochs=epochs,
    batch_size=batch_size,
    img_size=img_size,
    output_dir=output_dir,
    iou_threshold=iou_threshold,
    mosaic=mosaic,
    mixup=mixup
)


Loading model from /content/drive/MyDrive/yolov8s.pt
Training YOLOv8 with 200 epochs...
Ultralytics 8.3.78 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: task=detect, mode=train, model=/content/drive/MyDrive/yolov8s.pt, data=/content/dataset/data.yaml, epochs=200, time=None, patience=100, batch=64, imgsz=960, save=True, save_period=-1, cache=False, device=None, workers=8, project=runs_experiment, name=yolov8_training4, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.3, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_mas

train: Scanning /content/dataset/labels/train.cache... 2085 images, 0 backgrounds, 2 corrupt: 100%|██████████| 2085/2085 [00:00<?, ?it/s]

train: WARNING ⚠️ /content/dataset/images/train/4e6a8c0e-Ch120241207152724.png: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0725]
train: WARNING ⚠️ /content/dataset/images/train/b96d4520-Ch120241207124122.png: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0507]
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))



val: Scanning /content/dataset/labels/val.cache... 595 images, 0 backgrounds, 0 corrupt: 100%|██████████| 595/595 [00:00<?, ?it/s]


Plotting labels to runs_experiment/yolov8_training4/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000149, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 960 train, 960 val
Using 8 dataloader workers
Logging results to runs_experiment/yolov8_training4
Starting training for 200 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

      4/200      37.4G      1.387      1.253      1.398       1492        960: 100%|██████████| 33/33 [00:26<00:00,  1.26it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:09<00:00,  1.98s/it]


                   all        595      10279      0.775      0.783      0.824      0.501

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      5/200      36.5G      1.319      1.095      1.354       1577        960: 100%|██████████| 33/33 [00:21<00:00,  1.54it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:04<00:00,  1.08it/s]


                   all        595      10279      0.829      0.862      0.876      0.531

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      6/200      38.5G      1.272     0.9989      1.327       1306        960: 100%|██████████| 33/33 [00:22<00:00,  1.45it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  40%|████      | 2/5 [00:05<00:07,  2.62s/it]